In [1]:
from django.db import models
from cast.filters import PostFilterset

In [2]:
PostTag.objects.all()

<QuerySet [<PostTag: Weeknotes 2023-07-17 tagged with claas>, <PostTag: Weeknotes 2023-07-17 tagged with foobar>]>

In [2]:
tag = PostTag.objects.first()

In [7]:
tag.objects.all()

AttributeError: Manager isn't accessible via PostTag instances

In [5]:
blog = Blog.objects.last()

In [6]:
blog.unfiltered_published_posts.filter(tags__name="foobar")

<PageQuerySet [<Post: Weeknotes 2023-07-17>]>

In [13]:
Tag.objects.all()

<QuerySet [<Tag: claas>, <Tag: Nora>, <Tag: mastodon>, <Tag: screenshot>, <Tag: flechte>, <Tag: natur>, <Tag: baum>, <Tag: schwarz>, <Tag: katze>, <Tag: flugzeug>, <Tag: DUS>, <Tag: roller>, <Tag: badewanne>, <Tag: klettern>, <Tag: treppe>, <Tag: sipgate>, <Tag: spigate>, <Tag: nora>, <Tag: portrait>, <Tag: blume>, '...(remaining elements truncated)...']>

In [16]:
tag = PostTag.objects.first()

In [30]:
tag.tag

<Tag: foobar>

In [21]:
blog.unfiltered_published_posts.filter(tags__name__in=["foobar"])

<PageQuerySet []>

In [22]:
blog.unfiltered_published_posts.filter(tags__name="foobar")

<PageQuerySet []>

In [23]:
post = Post.objects.get(pk=392)

In [32]:
tag = post.tags.first()

In [34]:
blog.unfiltered_published_posts.filter(tags__in=[tag])

<PageQuerySet []>

In [33]:
tag

<Tag: claas>

In [35]:
Post.objects.filter(tags__in=[tag])

<PageQuerySet [<Post: Weeknotes 2023-07-17>]>

In [39]:
Post.objects.filter(tags__name="foobar")

<PageQuerySet [<Post: Weeknotes 2023-07-17>]>

In [40]:
blog.unfiltered_published_posts.filter(tags__name="foobar")

<PageQuerySet []>

In [37]:
Post.objects.all()

<PageQuerySet [<Post: Zu ersten Mal etwas Möhrenbrei>, <Post: Zu Besuch bei den Grosseltern>, <Post: Tapas mit Heike und Chrissie>, <Post: Martin und Marcus kommen zu Besuch>, <Post: Kaffeetrinken in der Spoerl-Fabrik>, <Post: In alten Möbeln>, <Post: Die U5 ist überstanden>, <Post: Spieldecke>, <Post: Schnullerversuch>, <Post: Ein Wochenende in Abentheuer>, <Post: Sofagebrabbel>, <Post: Claas zieht sich zum ersten Mal am Sofa hoch>, <Post: Zum ersten Mal Draussen>, <Post: Kinderwagen ist ok>, <Post: Erste Stehversuche>, <Post: Kurzbesuch bei Uroma Thea>, <Post: Zu Besuch in Meersen bei Onkel Jo>, <Post: Weihnachten 2017>, <Post: Mit Katharina bei Sulis>, <Post: Claas hält sich zum ersten Mal alleine am Tisch fest>, '...(remaining elements truncated)...']>

In [42]:
Post.objects.live().filter(tags__name="foobar")

<PageQuerySet [<Post: Weeknotes 2023-07-17>]>

In [43]:
blog

<Blog: Claas und Nora sagen Hallo>

In [44]:
PostTag.objects.all()

<QuerySet [<PostTag: Weeknotes 2023-07-17 tagged with claas>, <PostTag: Weeknotes 2023-07-17 tagged with foobar>]>

In [46]:
tag = PostTag.objects.first()

In [49]:
tag.tagged_items()

AttributeError: 'PostTag' object has no attribute 'tagged_items'

In [9]:
tag_count_queryset = PostTag.objects.annotate(
    num_posts=models.Count(
        "content_object",
        filter=models.Q(content_object__in=blog.unfiltered_published_posts)
    )
)

In [10]:
tag_count_queryset

<QuerySet [<PostTag: Weeknotes 2023-07-17 tagged with claas>, <PostTag: Weeknotes 2023-07-17 tagged with foobar>]>

In [12]:
tag_counts = {}
for tag in tag_count_queryset:
    tag_counts[tag.tag.name] = tag.num_posts  # type: ignore

In [13]:
tag_counts

{'claas': 1, 'foobar': 1}

In [14]:
set(PostTag.objects.values_list("tag__name", flat=True))

{'claas', 'foobar'}

In [6]:
post_tag = PostTag.objects.first()

In [8]:
post_tag.tag.slug

'claas'

In [3]:
set(PostCategory.objects.values_list("slug", flat=True))

{'fotos', 'til', 'video', 'weeknotes'}

# Facet Counts are wrong (categories instead of tags)

In [1]:
from django.db import models
from cast.filters import PostFilterset

In [2]:
blog = Blog.objects.get(slug="ephes_blog")
queryset = blog.unfiltered_published_posts

In [3]:
filterset = PostFilterset({}, queryset, fetch_facet_counts=True)

filterset cleaned_data:  {'search': '', 'date': None, 'category_facets': '', 'date_facets': '', 'o': None}
filterset filters:  OrderedDict([('search', <django_filters.filters.CharFilter object at 0x10d519b50>), ('date', <django_filters.filters.DateFromToRangeFilter object at 0x10d515690>), ('category_facets', <cast.filters.CategoryFacetFilter object at 0x10d5143d0>), ('date_facets', <cast.filters.DateFacetFilter object at 0x10d541590>), ('o', <django_filters.filters.OrderingFilter object at 0x10d0061d0>)])
category facet filter called with:  
show tag facet:  False


In [4]:
filterset.facet_counts["tags"]

{'claas': 1, 'foobar': 1}

In [5]:
tags_field = filterset.form.fields["tag_facets"]

tag choices:  [('claas', 'claas (1)'), ('foobar', 'foobar (1)')]


In [7]:
list(tags_field.choices)

[('', '---------'),
 ('fotos', 'fotos (1)'),
 ('til', 'til (1)'),
 ('weeknotes', 'weeknotes (2)')]

In [17]:
categories_field = filterset.form.fields["category_facets"]

In [18]:
list(categories_field.choices)

[('', '---------'),
 ('fotos', 'fotos (1)'),
 ('til', 'til (1)'),
 ('weeknotes', 'weeknotes (2)')]

In [5]:
dir(filterset)

['FILTER_DEFAULTS',
 'Meta',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_csv_filter_class_name',
 '_meta',
 'base_filters',
 'data',
 'declared_filters',
 'errors',
 'facet_counts',
 'filter_for_field',
 'filter_for_lookup',
 'filter_queryset',
 'filters',
 'form',
 'form_prefix',
 'fulltext_search',
 'get_fields',
 'get_filter_name',
 'get_filters',
 'get_form_class',
 'is_bound',
 'is_valid',
 'qs',
 'queryset',
 'request']

In [6]:
filterset._meta.fields

['search', 'date', 'date_facets', 'category_facets', 'tag_facets']

In [7]:
filterset.filters

OrderedDict([('search', <django_filters.filters.CharFilter at 0x10d0213d0>),
             ('date',
              <django_filters.filters.DateFromToRangeFilter at 0x10c0cf050>),
             ('category_facets',
              <cast.filters.CategoryFacetFilter at 0x10d50f750>),
             ('date_facets', <cast.filters.DateFacetFilter at 0x10d51ff10>),
             ('o', <django_filters.filters.OrderingFilter at 0x10d517090>)])

In [8]:
PageQuerySet

NameError: name 'PageQuerySet' is not defined